# journalists scraper

## Load Libraries

In [ ]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf
import urllib.request
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Functions

In [ ]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

## Functions Using Selenium

In [ ]:
# Define important paths
executable_path = "/chromedriver"
download_path = f"test"

# Define Routes (usually in xpaths)

def browser_instance(executable_path, headless, download_path):
    
    
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")   
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path})    
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver  

def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


In [ ]:
 driver = webdriver.Chrome("/Users/aadittambe/Documents/UMD/code_practice/news_app/test-stuff/chromedriver")
# driver.get("https://cpj.org/data/killed/?status=Killed&motiveConfirmed%5B%5D=Confirmed&type%5B%5D=Journalist&cc_fips%5B%5D=BG&cc_fips%5B%5D=BT&cc_fips%5B%5D=IN&cc_fips%5B%5D=MV&cc_fips%5B%5D=NP&cc_fips%5B%5D=PK&cc_fips%5B%5D=CE&start_year=1992&end_year=2021&group_by=year")

In [ ]:
def getter():
    counter = 0
    results = []
    driver.get("https://cpj.org/data/killed/?status=Killed&motiveConfirmed%5B%5D=Confirmed&type%5B%5D=Journalist&cc_fips%5B%5D=BG&cc_fips%5B%5D=BT&cc_fips%5B%5D=IN&cc_fips%5B%5D=MV&cc_fips%5B%5D=NP&cc_fips%5B%5D=PK&cc_fips%5B%5D=CE&start_year=1992&end_year=2021&group_by=year")
    while counter < 1:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[1]/div/nav/ul/li[8]/a')))
        table_finder = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[1]/div/table/tbody')))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup.prettify())
        table = soup.find(class_="table")
        rows = table.find_all('tr')
        for row in rows[1:]:
          cells = row.find_all('td')
          results.append([cell.text for cell in cells])
    #       print(results)
        
        next_button.click()
        print('debug')
        counter = counter + 1
        print(counter)
#         return(results)


        with open('results.csv','w') as output_file:
            csvfile = csv.writer(output_file)
            csvfile.writerow(['name', 'organization','date', 'location', 'death','type','na'])
            csvfile.writerows(results)
getter()

In [ ]:
# don't change
journalists_raw = pd.read_csv('journalists_killed.csv')
journalists_raw

In [139]:
# don't change
# def get_desc():
my_list = []
# providing url
end = 10
start = 1
names_list = []
url_list = []
desc_list = []
base_url = "https://cpj.org/data/people/"
for index, row in journalists_raw.iterrows():
    names_list.append(row["fullName"])
for name in names_list:
    name = name.replace(' ', '-')
    name_w_url = base_url + name
    url_list.append(name_w_url)
for link in url_list:
    url = link
    # opening the url for reading
    html = urllib.request.urlopen(url)

    # parsing the html file
    htmlParse = BeautifulSoup(html, 'html.parser')

    # getting all the paragraphs
    for para in htmlParse.find_all("p"):
        para = para.get_text()
    #     print(para)
        x = para.replace('Our EIN is 13-3081500.', '')
        x = x.replace('Committee to Protect Journalists', '')
        x = x.replace('P.O. Box 2675', '')
        x = x.replace('New York, NY 10108', '')
        x = x.replace('Tel 212-465-1004', '')
        x = x.replace('Fax 212-465-9568', '')
        x = x.replace('Except where noted, text on this website is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License.', '')
        x = x.replace('Images and other media are not covered by the Creative Commons license. For more information about permissions, see our FAQs.', '')
        x = x.replace('info@cpj.org', '')
        x = x.replace('CPJ is a 501(c)3 non-profit.', '')
        x = x.rstrip("\t")
    #     print(x)
        y = x.split('\n')
        for graf in y:
            my_list.append(graf)
    # print(my_list[0])
    res = my_list[: len(my_list) - 17]
    # print(res)
    popped_element = res.pop(0)
    # print(res)
    final_desc_clean = '<br> '.join(res)
    final_desc_clean
    desc_text = final_desc_clean
    desc_list.append(desc_text)
journalists_with_desc = journalists_raw
journalists_with_desc['desc'] = desc_list
journalists_with_desc
# get_desc()x

ValueError: Length of values (1) does not match length of index (169)

In [ ]:
# this is a step to just make sure
journalists_with_desc.to_csv('journalists.csv')

In [ ]:
# journalists with desc and location, country

journalists_city_country = journalists_with_desc
journalists_city_country["city_country"] = journalists_city_country["location"] + ", "+ journalists_city_country["country"]

journalists_city_country

In [ ]:
# clean that column using string detect
journalists_city_country_clean = journalists_city_country
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('an area outside ','')
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('an area near ','')
journalists_city_country_clean['city_country'] = journalists_city_country_clean['city_country'].str.replace('Malé','Male')


# journalists_city_country_clean.sort_values(by = ["city_country"], ascending=True)
journalists_city_country_clean

In [ ]:
jourcoords = journalists_city_country_clean #.to_csv("clean_journalists_deaths.csv")

In [ ]:
jourcoords = journalists_city_country_clean
jourcoords

In [ ]:
place_list = []
for place in jourcoords["city_country"]:
    place_list.append(place)
    
lat_list = []
long_list = []
for item in place_list:
#     while True:
    try: 
        geolocator = Nominatim(user_agent="Aadit")
        location = geolocator.geocode(item)
        lat_list.append(location.latitude)
        long_list.append(location.longitude)
    except (RuntimeError, TypeError, NameError, AttributeError):
        lat_list.append('pass')
        long_list.append('pass')

print(len(long_list))
print(len(lat_list))

In [ ]:
len(jourcoords)

In [ ]:
jourcoords["latitude"] = lat_list
jourcoords['longitude'] = long_list
jourcoords

In [ ]:
jourcoords.to_csv('final_file_clean.csv')

In [ ]:
# clean the descs
file = pd.read_csv('final_file.csv')
file["desc_clean"] = file["desc"].str.replace('\n','')
file["desc_clean"] = file["desc_clean"].str.replace('Share this:TwitterFacebookWhatsAppLinkedInEmailTelegram','')
file

file.to_csv('final_clean.csv')

In [ ]:
# importing modules

my_list = []
# providing url
end = 10
start = 1

url = "https://cpj.org/data/people/abadullah-hananzai/"

# opening the url for reading
html = urllib.request.urlopen(url)

# parsing the html file
htmlParse = BeautifulSoup(html, 'html.parser')

# getting all the paragraphs
for para in htmlParse.find_all("p"):
    para = para.get_text()
#     print(para)
    x = para.replace('Our EIN is 13-3081500.', '')
    x = x.replace('Committee to Protect Journalists', '')
    x = x.replace('P.O. Box 2675', '')
    x = x.replace('New York, NY 10108', '')
    x = x.replace('Tel 212-465-1004', '')
    x = x.replace('Fax 212-465-9568', '')
    x = x.replace('Except where noted, text on this website is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License.', '')
    x = x.replace('Images and other media are not covered by the Creative Commons license. For more information about permissions, see our FAQs.', '')
    x = x.replace('info@cpj.org', '')
    x = x.replace('CPJ is a 501(c)3 non-profit.', '')
    x = x.rstrip("\t")
#     print(x)
    y = x.split('\n')
    for graf in y:
        my_list.append(graf)
# print(my_list[0])
res = my_list[: len(my_list) - 17]
# print(res)
popped_element = res.pop(0)
# print(res)
final_desc_clean = '<br> '.join(res)
final_desc_clean

In [ ]:
x["test"] = x["desc_clean"].str.strip()

# x["test"] = x["test"].str.replace('\n','')
for i in x["test"]:
    print(i)
    print('--')

In [162]:
# don't change
# def get_desc():
my_list = []
# providing url
end = 10
start = 1
names_list = []
url_list = ["https://cpj.org/data/people/abadullah-hananzai/", "https://cpj.org/data/people/manish-kumar-singh/"]
for link in url_list:
    url = link
    # opening the url for reading
    html = urllib.request.urlopen(url)

    # parsing the html file
    htmlParse = BeautifulSoup(html, 'html.parser')
    
#     print(htmlParse)
#     print('--xxxx--')
    # getting all the paragraphs
    for para in htmlParse.find_all("p"):
        para = para.get_text()
#         print(para)
#     print('---')
        x = para.replace('Our EIN is 13-3081500.', '')
        x = x.replace('Committee to Protect Journalists', '')
        x = x.replace('P.O. Box 2675', '')
        x = x.replace('New York, NY 10108', '')
        x = x.replace('Tel 212-465-1004', '')
        x = x.replace('Fax 212-465-9568', '')
        x = x.replace('Except where noted, text on this website is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License.', '')
        x = x.replace('Images and other media are not covered by the Creative Commons license. For more information about permissions, see our FAQs.', '')
        x = x.replace('info@cpj.org', '')
        x = x.replace('CPJ is a 501(c)3 non-profit.', '')
        x = x.rstrip("\t")
        print(x)
        print('--')
#         y = x.split('\n')
#         for graf in y:
#             my_list.append(graf)
#     # print(my_list[0])
#     res = my_list[: len(my_list) - 17]
#     # print(res)
#     popped_element = res.pop(0)
#     # print(res)
#     final_desc_clean = '<br> '.join(res)
#     final_desc_clean
#     desc_text = final_desc_clean
#     desc_list.append(desc_text)
#     print(desc_list[0])


--
Abadullah Hananzai, a video journalist with U.S.-funded RFE/RL’s Radio Azadi, was killed in a double suicide bomb attack on April 30, 2018, in Kabul, Afghanistan, that left over 25 dead, among them at least nine journalists including Hananzai, RFE/RL reported. 
--
Hananzai was killed in the second blast, in which the attacker disguised himself as a media worker and detonated his explosives in a group of reporters, CPJ documented. 
--
Kabul police said the bomber deliberately targeted the press.
--
The extremist group Islamic State issued an online statement the same day that claimed responsibility for the attacks, which also injured at least 45 people, according to The Associated Press.
--
Hananzai, 26, had recently reported on drug addiction in Afghanistan and the Interior Ministry’s attempts to crackdown on international narcotics trafficking out of the country, according to RFE/RL. 
--
Hananzai was about to celebrate his first wedding anniversary, RFE/RL reported. His last publi